In [129]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [130]:
def encode_df(df):
    le = LabelEncoder()
    for col, col_type in df.dtypes.items():
        if col_type == 'object':
            df[col] = le.fit_transform(df[col])
    return df

def open_csv(filename):
    df = pd.read_csv('data/%s.csv' % filename, sep=';', low_memory=False)
    return df#encode_df(df)

def get_account_df():
    return open_csv('account').rename(columns={'date': 'account_date'})

def get_card_test_df():
    return open_csv('card_test').rename(columns={'type': 'card_type'})

def get_card_train_df():
    return open_csv('card_train').rename(columns={'type': 'card_type'})

def get_client_df():
    return open_csv('client')#.rename(columns={'district_id': 'client_district_id'})

def get_disp_df():
    return open_csv('disp').drop(['disp_id'], axis=1).rename(columns={'type': 'disp_type'})

def get_district_df():
    return open_csv('district').rename(columns={'code ': 'district_id'})

def get_loan_test_df():
    return open_csv('loan_test').rename(columns={'date': 'loan_date', 'amount': 'loan_amount'})

def get_loan_train_df():
    return open_csv('loan_train').rename(columns={'date': 'loan_date', 'amount': 'loan_amount'})

def get_trans_test_df():
    return open_csv('trans_test').rename(columns={'type': 'trans_type', 'date': 'trans_date', 'amount': 'trans_amount'})

def get_trans_train_df():
    return open_csv('trans_train').rename(columns={'type': 'trans_type', 'date': 'trans_date', 'amount': 'trans_amount'})


In [131]:
def merge_all(op='test'):
    account_df = get_account_df()
    card_df = get_card_test_df() if op == 'test' else get_card_train_df()
    client_df = get_client_df()
    disp_df = get_disp_df()
    district_df = get_district_df()
    loan_df = get_loan_test_df() if op == 'test' else get_loan_train_df()
    trans_df = get_trans_test_df() if op == 'test' else get_trans_train_df()

    ownership_df = disp_df.groupby("account_id", as_index=False, group_keys=False).agg({"disp_type": ["count"]})
    ownership_df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in ownership_df.columns]
    disp_df = pd.merge(disp_df, ownership_df, on='account_id')
    disp_df = disp_df[disp_df.disp_type == 'OWNER'].drop(['disp_type'], axis=1)

    df = pd.merge(loan_df, disp_df, on='account_id')
    df = pd.merge(df, client_df, on='client_id')
    df = pd.merge(df, district_df, on='district_id')
    df = pd.merge(df, account_df, on='account_id')
    df = pd.merge(df, trans_df, on='account_id')
    df.sort_values(by=["trans_date"], inplace=True)
    return df


merge_all('train').head(10)


,loan_id,account_id,loan_date,loan_amount,duration,payments,status,client_id,disp_type_count,birth_number,...,account_date,trans_id,trans_date,trans_type,operation,trans_amount,balance,k_symbol,bank,account
1988,6077,5270,931122,79608,24,3317,1,6367,1,700428,...,930113,1548749,930113,credit,credit in cash,800.0,800.0,NaN,NaN,NaN
1030,7284,11265,930915,52788,12,4399,1,13845,1,730216,...,930114,3393738,930114,credit,credit in cash,1000.0,1000.0,NaN,NaN,NaN
1989,6077,5270,931122,79608,24,3317,1,6367,1,700428,...,930113,1548750,930114,credit,collection from another bank,44749.0,45549.0,NaN,IJ,80269753.0
1574,7121,10364,931110,21924,36,609,1,12754,2,600820,...,930117,3122924,930117,credit,credit in cash,1100.0,1100.0,NaN,NaN,NaN
6977,5754,3834,940928,23052,12,1921,1,4620,2,406101,...,930119,1121963,930119,credit,credit in cash,700.0,700.0,NaN,NaN,NaN
18154,6895,9307,940919,41904,12,3492,1,11461,1,740708,...,930124,2809952,930124,credit,credit in cash,900.0,900.0,NaN,NaN,NaN
13441,6202,5891,940418,65184,12,5432,1,7127,1,570929,...,930125,1736607,930125,credit,credit in cash,900.0,900.0,NaN,NaN,NaN
15377,6316,6473,940526,76908,12,6409,-1,7815,1,465119,...,930126,1910780,930126,credit,credit in cash,1100.0,1100.0,NaN,NaN,NaN
15378,6316,6473,940526,76908,12,6409,-1,7815,1,465119,...,930126,1910790,930126,credit,credit in cash,19961.0,21061.0,NaN,NaN,NaN
454,5325,1843,930803,105804,36,2939,1,2235,1,405420,...,930130,541681,930130,credit,credit in cash,1000.0,1000.0,NaN,NaN,NaN
